In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

def output_results(out):
    pd.DataFrame({'deaths': out, 'Id': range(11)}).to_csv('mewmew_preds.csv', index=False)
    
def calculate_error(pred, real):
    return np.sqrt(np.mean((pred - real) ** 2))

training_df = pd.read_csv('phase1_training_data.csv')
training_df['date'] = training_df['date'].astype('datetime64')
training_df['country_id'].fillna('Unknown', inplace=True)

for col in training_df.columns[2:]:
    training_df[col+'_daily'] = training_df.groupby('country_id')[col].transform(lambda x: x.diff().fillna(0))
    
# ca_train_df = training_df[(training_df['country_id'] == 'CA') & (training_df['date'] >= '2020-03-15')]
ca_train_df = training_df[training_df['country_id'] == 'CA']

second_val_df = pd.read_csv('general_phase_1_answer.csv')
ca_val_df = second_val_df[second_val_df['country_id'] == 'CA']

In [10]:
ca_train_df

,country_id,date,cases,deaths,cases_14_100k,cases_100k,cases_daily,deaths_daily,cases_14_100k_daily,cases_100k_daily
8960,CA,2019-12-31,0,0,0.00,0.00,0,0,0.00,0.00
8961,CA,2020-01-01,0,0,0.00,0.00,0,0,0.00,0.00
8962,CA,2020-01-02,0,0,0.00,0.00,0,0,0.00,0.00
8963,CA,2020-01-03,0,0,0.00,0.00,0,0,0.00,0.00
8964,CA,2020-01-04,0,0,0.00,0.00,0,0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
9235,CA,2020-10-01,158758,9297,47.82,424.36,1797,6,1.81,4.80
9236,CA,2020-10-02,160535,9319,49.78,429.11,1777,22,1.96,4.75
9237,CA,2020-10-03,162659,9409,53.15,434.79,2124,90,3.37,5.68
9238,CA,2020-10-04,164471,9462,55.66,439.63,1812,53,2.51,4.84


In [2]:
new_training_df = pd.read_csv('general_phase_1_answer.csv')
new_training_df['country_id'].fillna('Unknown', inplace=True)
new_training_df = new_training_df[new_training_df['country_id'] == 'CA']

new_ca_train_df = pd.concat((ca_train_df, new_training_df[:-11]))

In [12]:
from AutoReg_MultiVar import AutoRegressionModelMultiVar as armv

xd = []

i = 0
for begining_ind in range(60, 120):
    data = new_ca_train_df[begining_ind:]
    deaths = data['deaths'].values
    cases = data['cases'].values
    for k_lag in range(1, 25):
        for seq_case_count in range(6):
            for case_lag in range(6, 26):
                lag_indices = list(np.arange(case_lag, min(case_lag+seq_case_count, 26)))
                model = armv(k=k_lag, bias=True)
                if len(lag_indices) == 0:
                    model.fit(deaths, np.empty(0), np.empty(0))
                else:
                    model.fit(deaths, [cases], [lag_indices])
                error = calculate_error(ca_val_df['deaths'][-11:-5].values, model.predict(len(data), len(data)+5))
                errs.append((begining_ind, k_lag, seq_case_count, case_lag, error))
                i += 1
                if i % 1000 == 0:
                    print("i is:", i)

i is: 1000
i is: 2000
i is: 3000
i is: 4000
i is: 5000
i is: 6000
i is: 7000
i is: 8000
i is: 9000
i is: 10000
i is: 11000
i is: 12000
i is: 13000
i is: 14000
i is: 15000
i is: 16000
i is: 17000
i is: 18000
i is: 19000
i is: 20000
i is: 21000
i is: 22000
i is: 23000
i is: 24000
i is: 25000
i is: 26000
i is: 27000
i is: 28000
i is: 29000
i is: 30000
i is: 31000
i is: 32000
i is: 33000
i is: 34000
i is: 35000
i is: 36000
i is: 37000
i is: 38000
i is: 39000
i is: 40000
i is: 41000
i is: 42000
i is: 43000
i is: 44000
i is: 45000
i is: 46000
i is: 47000
i is: 48000
i is: 49000
i is: 50000
i is: 51000
i is: 52000
i is: 53000
i is: 54000
i is: 55000
i is: 56000
i is: 57000
i is: 58000
i is: 59000
i is: 60000
i is: 61000
i is: 62000
i is: 63000
i is: 64000
i is: 65000
i is: 66000
i is: 67000
i is: 68000
i is: 69000
i is: 70000
i is: 71000
i is: 72000
i is: 73000
i is: 74000
i is: 75000
i is: 76000
i is: 77000
i is: 78000
i is: 79000
i is: 80000
i is: 81000
i is: 82000
i is: 83000
i is: 84000
i

In [15]:
error_df = pd.DataFrame(errs, columns=['begining_index', 'k_lag', 'case_sequence_count', 'case_lag', 'error'])
error_df

,begining_index,k_lag,case_sequence_count,case_lag,error
0,60,1,0,6,148.541031
1,60,1,0,7,148.541031
2,60,1,0,8,148.541031
3,60,1,0,9,148.541031
4,60,1,0,10,148.541031
...,...,...,...,...,...
345595,119,24,5,21,31.015882
345596,119,24,5,22,28.246702
345597,119,24,5,23,32.442471
345598,119,24,5,24,25.229916


In [24]:
error_df.sort_values('error', inplace=True)
error_df

,begining_index,k_lag,case_sequence_count,case_lag,error
301122,104,14,2,8,3.403316
292482,101,14,2,8,3.459849
298242,103,14,2,8,3.583537
315322,109,12,4,8,4.146083
306902,106,14,3,8,4.236381
...,...,...,...,...,...
2911,61,1,1,17,430.192325
2912,61,1,1,18,430.772750
33,60,1,1,19,431.121901
31,60,1,1,17,432.572610


In [61]:
error_df.to_csv('errors_2.csv', index=False)

In [25]:
error_df[error_df['error'] < 6]

,begining_index,k_lag,case_sequence_count,case_lag,error
301122,104,14,2,8,3.403316
292482,101,14,2,8,3.459849
298242,103,14,2,8,3.583537
315322,109,12,4,8,4.146083
306902,106,14,3,8,4.236381
...,...,...,...,...,...
334149,116,1,3,15,5.973510
316869,110,1,3,15,5.976255
219273,76,4,1,19,5.987090
331269,115,1,3,15,5.992839


In [56]:
for i in range(8, 20):
    print(i/2, sum(error_df['error'] < i/2))

4.0 3
4.5 7
5.0 29
5.5 146
6.0 334
6.5 651
7.0 1155
7.5 1741
8.0 2861
8.5 4383
9.0 5957
9.5 7575


In [20]:
data = ca_train_df[begining_ind:]
deaths = data['deaths'].values
cases = data['cases'].values

model = armv(k=2, bias=True)
lag_indices = list(np.arange(case_lag, min(case_lag+seq_case_count, 26)))
model.fit(data, )

6.296700513917712

In [21]:
output_results(model.predict(len(data), len(data)+10))

In [26]:
new_training_df = pd.read_csv('general_phase_1_answer.csv')
new_training_df['country_id'].fillna('Unknown', inplace=True)
new_training_df = new_training_df[new_training_df['country_id'] == 'CA']

new_ca_train_df = pd.concat((ca_train_df, new_training_df[:-5]))

In [1]:
test_errors = []
models = []
for ind, row in error_df[error_df['error'] <= 5].iterrows():
    data = new_ca_train_df[int(row.begining_index):]
    deaths = data['deaths'].values
    cases = data['cases'].values
    model = armv(k=int(row.k_lag), bias=True)
    lag_indices = list(np.arange(row.case_lag, min(row.case_lag+row.case_sequence_count, 26)).astype('int'))
    model.fit(deaths, [cases], [lag_indices])
    error = calculate_error(ca_val_df['deaths'][-5:].values, model.predict(len(data), len(data)+4))
    test_errors.append((row.begining_index, row.k_lag, row.case_sequence_count, row.case_lag, error))

NameError: name 'error_df' is not defined

In [58]:
test_error_df = pd.DataFrame(test_errors, columns=['begining_index', 'k_lag', 'case_sequence_count', 'case_lag', 'error'])
test_error_df['error'].mean()

28.402609191327223

In [59]:
test_error_df['error'].min()

4.337242210515835

In [60]:
preds = []
models = []
for ind, row in error_df[error_df['error'] <= 5.75].iterrows():
    data = new_ca_train_df[int(row.begining_index):]
    deaths = data['deaths'].values
    cases = data['cases'].values
    model = armv(k=int(row.k_lag), bias=True)
    lag_indices = list(np.arange(row.case_lag, min(row.case_lag+row.case_sequence_count, 26)).astype('int'))
    model.fit(deaths, [cases], [lag_indices])
    preds.append([model.predict(len(data), len(data)+4)])
    
calculate_error(np.median(np.array(preds).reshape(-1, 5), axis=0), ca_val_df['deaths'][-5:].values)

18.94074624313744